In [ ]:
site = "m1s1"
model_name = "rf_cnn"
topk = 10
print_on = True

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
from torch.utils.data import Dataset,DataLoader
from training.utils import *

from modeling import *

val_img = np.load(f'/val_img_{site}.npy')
val_resp = np.load(f'/valRsp_{site}.npy')
val_img = np.reshape(val_img,(-1,1,50,50))

net = model_rf_cnn_one_neuron()
        

In [ ]:
# corr
corrs = []
for neuron_number in range(val_resp.shape[1]):
    net.load_state_dict(torch.load(f'../results/{site}_model_{model_name}/neuron_{neuron_number}.pth', map_location=torch.device('cpu')))
    corr = pc_one_neuron(net,neuron_number,val_img,val_resp)
    if print_on: print(corr)
    corrs.append(corr)

In [ ]:
# pt_j
for neuron_number in range(val_resp.shape[1]):
    net.load_state_dict(torch.load(f'../results/{site}_model_{model_name}/neuron_{neuron_number}.pth', map_location=torch.device('cpu')))
    with torch.no_grad():
        net.eval()
        pred = net(torch.from_numpy(np.float32(val_img))).detach().numpy().T[0]
    real = val_resp[:,neuron_number]
    zipped = list(zip(list(range(1000)),pred,real))
    zipped.sort(key=lambda x:x[2], reverse=True)

    top_pred = np.asarray([a[1] for a in zipped][:topk])
    top_real = np.asarray([a[2] for a in zipped][:topk])
    threshold, members = top_real[-1], 0
    for p in top_pred:
        if p>= threshold: members+=1
    if print_on: print(members/topk)


In [ ]:
# pt_s
for neuron_number in range(val_resp.shape[1]):
    net.load_state_dict(torch.load(f'../results/{site}_model_{model_name}/neuron_{neuron_number}.pth', map_location=torch.device('cpu')))
    with torch.no_grad():
        net.eval()
        pred = net(torch.from_numpy(np.float32(val_img))).detach().numpy().T[0]
    real = val_resp[:,neuron_number]
    zipped = list(zip(list(range(1000)),pred,real))
    zipped.sort(key=lambda x:x[2], reverse=True)

    top_pred = np.asarray(sorted([a[1] for a in zipped],reverse=True)[:topk])
    top_real = np.asarray(sorted([a[2] for a in zipped],reverse=True)[:topk])
    threshold, members = top_real[-1], 0
    for p in top_pred:
        if p>= threshold: members+=1
    if print_on: print(members/topk)
        


In [ ]:
# tuning curves
savepathname = f'./tuning_curves/{site}_model_{model_name}'
for neuron_number in range(val_resp.shape[1]):
    net.load_state_dict(torch.load(f'../results/{site}_model_{model_name}/neuron_{neuron_number}.pth', map_location=torch.device('cpu')))
    with torch.no_grad():
        net.eval()
        pred = net(torch.from_numpy(np.float32(val_img))).detach().numpy().T[0]
    real = val_resp[:,neuron_number]
    zipped = list(zip(list(range(1000)),pred,real))
    zipped.sort(key=lambda x:x[2], reverse=True)
    plt.plot(list(range(1000)),[x[2] for x in zipped], label = "Real Responses")
    plt.plot(list(range(1000)),[x[1] for x in zipped], label = "Predicted Responses")
    plt.xlabel("Image (Rank Ordered by Real Response)")
    plt.ylabel("Neural Response")
    plt.title(f'Site: {site}, Model: {model_name}, Neuron: {neuron_number}')
    plt.legend()
    plt.savefig(f'{savepathname}/neuron_{neuron_number}')
    if print_on: plt.show()
    plt.clf()

In [ ]:
# average tuning curve
savepathname = f'./tuning_curves/{site}_model_{model_name}'
accum_pred = [[] for x in range(1000)]
accum_real = [[] for x in range(1000)]
for neuron_number in range(val_resp.shape[1]):
    net.load_state_dict(torch.load(f'../results/{site}_model_{model_name}/neuron_{neuron_number}.pth', map_location=torch.device('cpu')))
    with torch.no_grad():
        net.eval()
        pred = net(torch.from_numpy(np.float32(val_img))).detach().numpy().T[0]
    real = val_resp[:,neuron_number]
    zipped = list(zip(list(range(1000)),pred,real))
    zipped.sort(key=lambda x:x[2], reverse=True)
    for i in range(1000):
        accum_pred[i].append(zipped[i][1])
        accum_real[i].append(zipped[i][2])
avg_pred = [np.mean(x) for x in accum_pred]
avg_real = [np.mean(x) for x in accum_real]
plt.plot(list(range(1000)),avg_real, label = "Average Real Responses")
plt.plot(list(range(1000)),avg_pred, label = "Average Predicted Responses")
plt.xlabel("Image (Rank Ordered by Real Response)")
plt.ylabel("Average Neural Response")
plt.title(f'Site: {site}, Model: {model_name}, Average')
plt.legend()
plt.savefig(f'{savepathname}/average')
if print_on: plt.show()
plt.clf()

